# Prueba de las IA construidas
### Curso 2021 - Programación Lógica - UdelaR
Este notebook es una ayuda para simular varias partidas de Arabelog.



In [1]:
# En Prolog funciona.
# Veamos ahora si podemos exteder a Python
#from os import path
from typing import List, Optional
from itertools import islice

import prolog_bridge as bridge
import importlib
import re
import random
import time


### Funciones auxiliares

In [2]:

def contar_piezas(tablero,jugador):
    return [x for fila in tablero for x in fila].count(jugador)

def mostrar_estado(tablero,jugador_actual,datos_estado,piezas,fase):
    print(jugador_actual,piezas)
    print(bridge.tablero_to_prolog(tablero,datos_estado,jugador_actual))


### generar_tablero()
La función generar_tablero() genera un tablero vacío y agrega las primeras 12 piezas de forma aleatoria

In [3]:

def generar_tablero():
    tablero = [
        ['', '' , '' , '', ''],
        ['', '', '', '', ''],
        ['', '', '', '', ''],
        ['', '', '', '', ''],
        ['', '', '' , '','']]
    
    libres=[(1,1),(1,2),(1,3),(1,4),(1,5),
            (2,1),(2,2),(2,3),(2,4),(2,5),
            (3,1),(3,2),(3,4),(3,5),
            (4,1),(4,2),(4,3),(4,4),(4,5),
            (5,1),(5,2),(5,3),(5,4),(5,5),
           ]
    
    # Ubicamos la mitad de las piezas al azar
    # para que los juegos no sean siempre iguales
    random.shuffle(libres)
    libres=libres[0:12]
    jugador='X'
    for (fila,col) in libres: 
        tablero[fila-1][col-1]=jugador
        if jugador=='X':
            jugador='O'
        else:
            jugador='X'

    return tablero



        

### partida(...)

A partir de un tablero inicial, y dos jugadores, cada uno con su estrategia, ejecuta una partida. Devuelve el ganador (o empate), y las piezas que le quedaron a cada jugador en el tablero

In [4]:
def partida(tablero_inicial,fase_inicial,estrategiaX,minimaxX,estrategiaO,minimaxO): 
    
    
    # Estado inicial inicial
    #tablero = generar_tablero()
    tablero= tablero_inicial
    fase=fase_inicial
    
    # Los datos del estado son: 
    # Veces seguidas que pasó X
    # Veces seguidasque pasó O
    # Veces sin capturar de X
    # Veces sin capturar de O
    # Fase en la que estamos jugando

    piezas={'X':0, 'O':0}
    piezas['X']=contar_piezas(tablero,'X')
    piezas['O']=contar_piezas(tablero,'O')


    pasadas= {'X':0, 'O':0}
    sin_captura={'X':0, 'O':0}
    estrategia={'X':estrategiaX,'O':estrategiaO}
    minimax={'X':minimaxX,'O':minimaxO}

    fin = False

    # Comienza X
    jugador_actual='X'
    rival='O'    

    
    while not fin: 
        datos_estado=[pasadas['X'],pasadas['O'],sin_captura['X'],sin_captura['O'],fase]

        #mostrar_estado(tablero,jugador_actual,datos_estado,piezas,fase)
    
        if (fase==1):
            # Lo unico que tengo que hacer es insertar dos piezas
            tablero=bridge.do_mejor_movimiento(tablero,datos_estado,jugador_actual,
                                               minimax[jugador_actual] ,estrategia[jugador_actual])
            
            piezas['X']=contar_piezas(tablero,'X')
            piezas['O']=contar_piezas(tablero,'O')

            # Veo si no tiene que pasar a la fase 2
            if piezas[jugador_actual]==12 and piezas[rival]==12:
                fase=2
        else:
            # Estoy en fase 2
            # Antes de jugar cuento las piezas del rival
            piezas_rival_antes=piezas[rival] 
    
            # Antes de romper todo, veo si puede jugar
            if bridge.do_hay_movimiento(tablero,datos_estado,jugador_actual):
                # Elijo el mejor movimiento y lo ejecuto
                tablero=bridge.do_mejor_movimiento(tablero,datos_estado,jugador_actual,
                                               minimax[jugador_actual] ,estrategia[jugador_actual])

                piezas['X']=contar_piezas(tablero,'X')
                piezas['O']=contar_piezas(tablero,'O')

                # Veo si no ganó
                # Eso sucedería si ya no quedan piezas del rival
                if piezas[rival]==0: 
                    fin = True
                    ganador=jugador_actual

                # Reseteo la cantidad de veces que pasó 
                pasadas[jugador_actual]=0

                # Veo si comió y actualizo la variable correspondiente
                if piezas[rival]< piezas_rival_antes:
                    sin_captura[jugador_actual]=0
                else:
                    sin_captura[jugador_actual]+=1
            else:
                # No pudo jugar
                pasadas[jugador_actual]+=1

                # Si es la tercera seguida, entonces pierde
                if pasadas[jugador_actual]==3:
                    fin=True
                    ganador=rival
    
            # Controlo empate
            if sin_captura['X']>=12 and sin_captura['O']>=12:
                fin=True
                ganador='Empate'
    
        
        # Cambio el jugador actual
        if jugador_actual=='X':
            jugador_actual='O'
            rival='X'
        else:
            jugador_actual='X'
            rival='O'
            
            
    # Cuando termina, muestro el ganador
    return (ganador,piezas['X'],piezas['O'])



### Probar juego

Creamos el tablero y ejecutamos una partida

In [5]:
# Este lo usamos para probar un juego con un tablero particular

tablero = [
        ['X', 'O' ,'X' , 'O', 'O'],
        ['O', 'X', 'O' , 'X', 'O'],
        ['O', 'X',  '' , 'X', 'X'],
        ['X', 'X', 'O' , 'X', 'O'],
        ['O', 'X', 'X' , 'O', 'O']]

(ganador,piezasX,piezasO)=partida(tablero,2,'minimax',3,'dummy',0)

print((ganador,piezasX,piezasO))

hay_movimiento(estado(m(f(x,o,x,o,o),f(o,x,o,x,o),f(o,x,-,x,x),f(x,x,o,x,o),f(o,x,x,o,o)),0,0,0,0,2),x).
mejor_movimiento(estado(m(f(x,o,x,o,o),f(o,x,o,x,o),f(o,x,-,x,x),f(x,x,o,x,o),f(o,x,x,o,o)),0,0,0,0,2),x,3,minimax,Estado2).
('X', 0, 0)


### Simulación

Ejecuta n partidas entre dos jugadores con diferentes estrategias y muestra los resultados

In [ ]:
# Las partidas son aleatorias, pero siempre las mismas. Para probar.
random.seed(25)

ganaX=0
ganaO=0
empateX=0
empateO=0
empatan=0
total_juegos=12


start = time.time()
for p in range(total_juegos):
    tablero_inicial=generar_tablero()
    #print('Tablero inicial:',bridge.tablero_to_prolog(tablero_inicial,[0,0,0,0,0],'X'))

    (ganador,piezasX,piezasO)=partida(tablero_inicial,2,'ia_grupo',5,'ia_grupo',3)
    if ganador=='Empate':
        empatan += 1
        
        if piezasX > piezasO:
            empateX += 1
            print('Empate X',piezasX,piezasO)

        elif piezasX < piezasO:
            empateO += 1
            print('Empate O',piezasX,piezasO)

        else:
            None
            print('Empate ',ganador,piezasX,piezasO)
    elif ganador=='X':
        print('Ganó ',ganador,piezasX,piezasO)
        ganaX +=1
    else:
        print('Ganó ',ganador,piezasX,piezasO)
        ganaO +=1

pganaX = ganaX/total_juegos
pganaO = ganaO/total_juegos
pempateX = empateX/total_juegos
pempateO = empateO/total_juegos
pempatan = empatan/total_juegos

empatanE = empatan - empateX - empateO

end = time.time()
elapsed=end-start

print(f'Victorias X:{ganaX} ({pganaX}) Victorias O:{ganaO}({pganaO}) Empates:{empatan}({pempatan}) [{empateX}-{empateO}-{empatanE}]')
print(f'Tiempo:{elapsed}')

hay_movimiento(estado(m(f(-,-,o,x,x),f(o,-,o,-,-),f(o,o,-,-,x),f(o,-,-,x,-),f(-,-,-,x,x)),0,0,0,0,2),x).
mejor_movimiento(estado(m(f(-,-,o,x,x),f(o,-,o,-,-),f(o,o,-,-,x),f(o,-,-,x,-),f(-,-,-,x,x)),0,0,0,0,2),x,5,ia_grupo,Estado2).
hay_movimiento(estado(m(f(-,-,o,-,x),f(o,-,o,x,-),f(o,o,-,-,x),f(o,-,-,x,-),f(-,-,-,x,x)),0,0,1,0,2),o).
mejor_movimiento(estado(m(f(-,-,o,-,x),f(o,-,o,x,-),f(o,o,-,-,x),f(o,-,-,x,-),f(-,-,-,x,x)),0,0,1,0,2),o,3,ia_grupo,Estado2).
hay_movimiento(estado(m(f(-,o,-,-,x),f(o,-,o,x,-),f(o,o,-,-,x),f(o,-,-,x,-),f(-,-,-,x,x)),0,0,1,1,2),x).
mejor_movimiento(estado(m(f(-,o,-,-,x),f(o,-,o,x,-),f(o,o,-,-,x),f(o,-,-,x,-),f(-,-,-,x,x)),0,0,1,1,2),x,5,ia_grupo,Estado2).
hay_movimiento(estado(m(f(-,o,-,x,-),f(o,-,o,x,-),f(o,o,-,-,x),f(o,-,-,x,-),f(-,-,-,x,x)),0,0,2,1,2),o).
mejor_movimiento(estado(m(f(-,o,-,x,-),f(o,-,o,x,-),f(o,o,-,-,x),f(o,-,-,x,-),f(-,-,-,x,x)),0,0,2,1,2),o,3,ia_grupo,Estado2).
hay_movimiento(estado(m(f(o,-,-,x,-),f(o,-,o,x,-),f(o,o,-,-,x),f(o,-,-,x,-),